# ABM

## Imports / Data Conversion

In [1]:
import time, enum, math
import numpy as np
import pandas as pd
import pylab as plt
from mesa import Agent, Model
from mesa.time import SimultaneousActivation, RandomActivation
from mesa.space import NetworkGrid
from mesa.datacollection import DataCollector
from networkx.algorithms.shortest_paths.generic import has_path
import networkx as nx
import panel as pn
import panel.widgets as pnw
import random
from tqdm import tqdm, trange
from time import sleep

data_path = '../' #set to wherever the data files are, will be used on every input

In [2]:
ports = pd.read_csv((data_path +'ports.csv'))


origin = pd.read_csv((data_path + 'origin_ports.csv'))
clean_distances = pd.read_csv((data_path + 'clean_distances.csv'))
# distances = distances_df[["prev_port", "next_port", "distance"]]
# distances.astype({'prev_port':'int64', 'next_port':'int64'}).dtypes

In [3]:
route_blockage_dov = pd.read_csv((data_path + 'route_blockages_dov.csv'))
route_blockage_gib = pd.read_csv((data_path + 'route_blockages_gib.csv'))
route_blockage_horm = pd.read_csv((data_path + 'route_blockages_horm.csv'))
route_blockage_mal = pd.read_csv((data_path + 'route_blockages_mal.csv'))
route_blockage_pan = pd.read_csv((data_path + 'route_blockages_pan.csv'))
route_blockage_suez = pd.read_csv((data_path + 'route_blockages_suez.csv'))
route_blockage_total = pd.read_csv((data_path + 'route_blockages_total.csv'))

In [4]:
pruning_files = [route_blockage_dov, route_blockage_gib, route_blockage_horm, route_blockage_mal, route_blockage_pan, route_blockage_suez, route_blockage_total]

In [8]:
def Cut_Graph(G, route_blockages):
        fails = 0
        for index in range(len(route_blockages)):
            try:
                G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
            except:
                fails += 1
        print("Failed {} times".format(fails))
        return G

## Playground 

In [5]:
G = nx.from_pandas_edgelist(clean_distances, "prev_port", "next_port",edge_attr= "distance",create_using=nx.Graph())

In [6]:
G.number_of_edges()

29838

In [7]:
G.number_of_nodes()

2912

In [44]:
G_Total.number_of_edges()

24306

In [43]:
G_Total = Cut_Graph(G, route_blockage_total)

Failed 1930 times


In [55]:
list(G.nodes())[1]

1940

In [63]:
results = []
Success = 0
Fail = 0
Same_distance = 0
for i in range(len(list(G.nodes()))):
    position = list(G.nodes())[i]
        for j in range(len(list(G.nodes()))):
            destination = list(G.nodes())[i]
            try:
                init_route = nx.dijkstra_path(G, position, destination, weight='distance')
                init_dist = nx.dijkstra_path_length(G, position, destination, weight='distance')
                Total_route = nx.dijkstra_path(G_Total, position, destination, weight='distance')
                Total_dist = nx.dijkstra_path_length(G_Total, position, destination, weight='distance')
                if init_dist != Total_dist:
                    result = [position, destination]
                    results.append(result)
                    Success += 1
                else:
                    Same_distance += 1
            except :
                Fail += 1

        

IndentationError: unexpected indent (<ipython-input-63-87969c51f070>, line 7)

In [61]:
print(Success, Same_distance, Fail)

0 757 2155


In [12]:
init_route = nx.dijkstra_path(G, position, destination, weight='distance')
init_dist = nx.dijkstra_path_length(G, position, destination, weight='distance')
Total_route = nx.dijkstra_path(G_Suez, position, destination, weight='distance')
Total_dist = nx.dijkstra_path_length(G_Suez, position, destination, weight='distance')

In [14]:
init_dist

6063.350807988509

In [ ]:
# def next_dest(sample):
#     try:

#         overall_distance = list()
#         itinerary = list()
#         for i in range(len(sample)):
#             distance = dict()
#             for i in range(1,len(sample)):
#                 distance[sample[i]] = nx.dijkstra_path_length(G, sample[0] ,sample[i], weight='distance')
#             next_stop = min(distance, key=distance.get)
#             itinerary.append(nx.dijkstra_path(G, origin, next_stop, weight = 'distance'))
#             overall_distance.append(next_stop)
#             sample.pop(0)
#             ind = sample.index(next_stop)
#             sample.pop(ind)
#             sample.insert(0, next_stop)
#     except:
#         pass
#     flat_route = []
#     for sublist in itinerary:
#         for port in sublist:
#             flat_route.append(port)
        
#     return flat_route, sum(overall_distance)


## Model

In [39]:
class ShippingNetwork(Model):
    def __init__(self, distances, major_ports, pruning_files, S=1000):
        self.major_ports = major_ports
        self.num_ships = S
        self.distances = distances
        self.schedule = SimultaneousActivation(self)
        self.running = True
        self.Ships = []
        self.pruning_files = pruning_files

        #Build Network without closures
        self.G = nx.from_pandas_edgelist(distances, "prev_port", "next_port", ["distance"], create_using=nx.Graph())    
        #Define Mesa Grid as the just created Network to allow for shipping only in routes
        self.grid = NetworkGrid(self.G) 
    

        #Build alternate Networks (with closures in place)
        self.G_Dov = self.Cut_Graph(self.G, self.pruning_files[0])





        #create agents   
        for i in tqdm(range(self.num_ships), desc="Placing Ships"):
            start_port = np.random.choice(self.major_ports["Ref"], size=None, replace=True,  p=self.major_ports["PROB"])
            destin_port = random.sample(self.G.nodes, k=1)[0]  # Sample the destination 
            #We sample the origin port from a list of the 50 biggest ports world, with the prob = TAU of the port / TAU of all origin ports for 2017
            a = Ship(i+1, self, self.G, start_port, destin_port, major_ports)
            self.schedule.add(a)
            self.Ships.append(a) #append to list of ships
            self.grid.place_agent(a, start_port) #place agent on origin node

        self.datacollector = DataCollector(
            model_reporters={"Graph":"G"},
            agent_reporters={"Position": "position", "Destination":"destination", "Itinerary":"itinerary", "Distance_Traveled":"distance_traveled", "Route":"current_route", "Route Changes":"route_chng" })
    


    def Cut_Graph(self, G, route_blockages):
        for index in range(len(route_blockages)):
            try:
                G.remove_edge(route_blockages.iloc[index]['prev_port'],route_blockages.iloc[index]['next_port'])
            except:
                pass
        return G


        #create ability to remove edges mid-model
    def network_change(self, blockage):
        if blockage == "Dover":
            for a in tqdm(self.Ships):
                a.G = self.G_Dov
            print("done changing, should step again?")


    def step(self, blockage=''):
        #check network for changes
        if blockage != '' :
           network_change(blockage)

        self.schedule.step()     #Run each Agents
        self.datacollector.collect(self)




In [40]:
class Ship(Agent):
    def __init__(self, unique_id, model, G, start_port, destin_port, major_ports, s=13.0):
        super().__init__(unique_id, model)
        self.destination = destin_port
        self.state = 0 #0 for active, numbers > 0 for weeks that ships have to "wait" until arrival to port
        self.speed = s*24*1.852 #speed is given in knots, with 1 knot being 1 nautical mile per hour. Since the model works with distances in km, we convert here (1 nm = 1.852m)
        self.position = start_port
        self.distance_traveled = 0
        self.unique_id = unique_id
        self.G = G
        self.route_chng = 0
        self.routes = 0
        self.major_ports = major_ports

    
        #implement dijkstra to define shortest route
        try:
            self.init_route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            self.init_dist = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        except:
            self.destination = random.sample(self.G.nodes, k=1)[0]
            self.init_route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            self.init_dist = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
            #include a way for capacity?
        

         #We keep a copy of the entire itinerary / distance traveled
        self.current_route, self.current_dist = self.init_route, self.init_dist  #For comparison & navigational purposes, we use current route & distance
        self.next_position = self.current_route[1]
        self.itinerary = [self.position]
        self.step_size = self.ident_distance()


    def routing(self):
        #implement dijkstra to define shortest route
        try:
            route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            travel_distance = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        except:
            self.destination = random.sample(self.G.nodes, k=1)[0]
            route = nx.dijkstra_path(self.G, self.position, self.destination, weight='length')
            travel_distance = nx.dijkstra_path_length(self.G, self.position, self.destination, weight='length')
        #include a way for capacity?
        return route, travel_distance


    def move(self):
        self.next_position = self.current_route[1]
        self.step_size = self.ident_distance() #look up the distance between two cities 
        self.state = self.step_size / self.speed #change state to step amount
        self.current_dist = self.current_dist - self.step_size #adjust current distance minus the distance traveled in the next step
        self.model.grid.move_agent(self, self.next_position) #move the agent
        self.current_route.remove(self.current_route[0]) #remove the next step from the itinerary
        self.position = self.next_position
        if len(self.current_route) == 1:
            self.next_position = self.current_route[0] 
        else:
            self.next_position = self.current_route[1] #update current route

    def ident_distance(self): #look up the distance of the current step
        return self.G.get_edge_data(self.position, self.next_position)['distance']
    
    def reroute(self):
        self.routes += 1
        if self.position in self.major_ports["Ref"].values: # We re-route in such a way that ships that have either an origin port or a destination port that is a major port
            self.destination = random.sample(self.G.nodes, k=1)[0]
        else:
            self.destination = np.random.choice(self.major_ports["Ref"], size=None, replace=True,  p=self.major_ports["PROB"])
        
        self.current_route, self.current_dist = self.routing()
        self.state = 3

    

    def step(self):
        self.state = self.state - 1 #'move' ships by one weeks progress
        if self.state <= 0.000: #ships that are en-route to the node they are going to next do not move / perform other activities
            self.distance_traveled += self.step_size #ship has arrived at port, let's add the distance traveled to their 
           
            #add the current position to itinerary
            if self.position != self.destination: #if current stop is not the final stop
                new_route, new_distance = self.routing() #perform a new routing to compare against current routing
                
                if new_route == self.current_route: #if current routing is the same as new, just move (default case)
                    # print("default case")
                    self.move()
                    self.itinerary.append(self.position)
        
            # THIS CURRENTLY ONLY CHANGES THE ROUTE IF THE NEXT STEP IS BLOCKED
                elif new_distance > self.current_dist: #if current route is shorter than newly calculated route, check for obstructions
                    # print('reroute: current route shorter than new route')
                    if not has_path(self.G, self.position, self.next_position): 
                        self.current_route = new_route
                        self.current_dist = new_distance
                        self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
                
                
                else: # final option is that current route is longer than new route (think Suez reopening after a while), here, we just take the new option
                    # print('reroute: current route longer than new route')
                    self.current_route = new_route
                    self.current_dist = new_distance
                    self.route_chng += 1
                    self.move()
                    self.itinerary.append(self.position)
            
            else: #if ship is arrived at final position, get a new route, and start back
                # print('arrival')
                self.reroute()
                self.itinerary.append(self.position)
        # print("Ship: {}, Source: {}, Destination: {}, Position: {},  Next Stop {}, Time until next Stop {}".format(self.unique_id, self.itinerary[0], self.destination, self.position, self.next_position, self.state ))

    
    # def collect_time

    # def collect_costs

## Model Instances

In [42]:

model = ShippingNetwork(clean_distances, origin, pruning_files, 1)

steps = 6
for i in trange(steps, desc="Stepping Model"):
    model.step()
model.network_change('Dover')
for i in trange(steps, desc="Stepping Model after changes"):
    model.step()
print("does this execute here?")


agent_state = model.datacollector.get_agent_vars_dataframe()
agent_state

Stepping Model after changes: 100%|██████████| 6/6 [00:00<00:00, 14090.61it/s]done changing, should step again?
does this execute here?



,,Position,Destination,Itinerary,Distance_Traveled,Route,Route Changes
Step,AgentID,,,,,,
1,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0
2,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0
3,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0
4,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0
5,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0
6,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0
7,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0
8,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0
9,1,1975,42159,"[2403.0, 1975]",7614.344734,"[1975, 2172, 42159]",0


,,Position,Destination,Itinerary,Distance_Traveled,Route,Route Changes
Step,AgentID,,,,,,
1,1,2667,1766,"[2088.0, 2667, 3099]",227.108845,"[3099, 4451, 1766]",0
2,1,3099,1766,"[2088.0, 2667, 3099]",454.217691,"[3099, 4451, 1766]",0
3,1,3099,1766,"[2088.0, 2667, 3099]",454.217691,"[3099, 4451, 1766]",0
4,1,3099,1766,"[2088.0, 2667, 3099]",454.217691,"[3099, 4451, 1766]",0
5,1,3099,1766,"[2088.0, 2667, 3099]",454.217691,"[3099, 4451, 1766]",0
6,1,3099,1766,"[2088.0, 2667, 3099]",454.217691,"[3099, 4451, 1766]",0
7,1,3099,1766,"[2088.0, 2667, 3099]",454.217691,"[3099, 4451, 1766]",0


In [68]:
agent_state.to_csv('../test_out.csv')

138038.67166337415